In [1]:
import string
from itertools import islice

In [2]:
import pandas as pd
import numpy as np

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer



In [5]:
docA = """A expressão inteligência emocional está incluída hoje no léxico de muitos, 
tanto de pessoas comuns quanto de intelectuais ou celebridades. Até mesmo ministros usam a 
expressão em seus comentários e alertas. Mas nem todo mundo se refere à mesma coisa quando usa essa expressão. 
Para alguns, a inteligência emocional é algo como um tipo de inteligência mais avançada do que a clássica, ou seja, 
do que a inteligência analítica, medida em testes que fornecem resultados em quociente numérico.
Há também aqueles que se referem à inteligência emocional pelo lado negativo, 
como uma incapacidade de controlar as emoções: Comporta-se como se não tivesse inteligência emocional. 
Também não faltam aqueles que acreditam ser um novo tipo de inteligência recém-inventada, pois, afinal de contas, 
o conceito de inteligência não é absoluto, tal como a altura ou o peso de uma pessoa, 
pois sempre depende do critério do observador. Outros, por sua vez, 
sequer sabem a que se referem quando falam sobre esse tipo de inteligência. 
Talvez, por tudo isso, valha a pena tentar esclarecer o conceito."""

In [6]:
docA

'A expressão inteligência emocional está incluída hoje no léxico de muitos, \ntanto de pessoas comuns quanto de intelectuais ou celebridades. Até mesmo ministros usam a \nexpressão em seus comentários e alertas. Mas nem todo mundo se refere à mesma coisa quando usa essa expressão. \nPara alguns, a inteligência emocional é algo como um tipo de inteligência mais avançada do que a clássica, ou seja, \ndo que a inteligência analítica, medida em testes que fornecem resultados em quociente numérico.\nHá também aqueles que se referem à inteligência emocional pelo lado negativo, \ncomo uma incapacidade de controlar as emoções: Comporta-se como se não tivesse inteligência emocional. \nTambém não faltam aqueles que acreditam ser um novo tipo de inteligência recém-inventada, pois, afinal de contas, \no conceito de inteligência não é absoluto, tal como a altura ou o peso de uma pessoa, \npois sempre depende do critério do observador. Outros, por sua vez, \nsequer sabem a que se referem quando fala

In [7]:
tokenizer = RegexpTokenizer(r'\w+')
bowA = tokenizer.tokenize(docA)


In [9]:
stpwrds = stopwords.words('portuguese') + list(string.punctuation)

In [10]:
#stpwrds

#### Retirando os StopWords

In [11]:
bowA = ([w.lower() for w in bowA if w not in stpwrds])

In [12]:
#bowA

In [13]:
# CountVectorizer/TfidfTransformer
cvec = CountVectorizer( min_df=1, max_df=.5, ngram_range=(1,2))
cvec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [14]:
cvec.fit(bowA)
list(islice(cvec.vocabulary_.items(), 10))

[('expressão', 24),
 ('inteligência', 33),
 ('emocional', 21),
 ('incluída', 31),
 ('hoje', 28),
 ('léxico', 36),
 ('muitos', 41),
 ('tanto', 68),
 ('pessoas', 52),
 ('comuns', 15)]

In [15]:
len(cvec.vocabulary_)

78

In [16]:
cvec_counts = cvec.transform(bowA)
print('Matriz esparsa :', cvec_counts.shape)
print('Nonzero count:', cvec_counts.nnz)
print('Sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1]))) 

Matriz esparsa : (99, 78)
Nonzero count: 96
Sparsity: 1.24%


In [17]:
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'Palavra': cvec.get_feature_names(), 'Ocorrências': occ})
counts_df.sort_values(by='Ocorrências', ascending=False).head(5)

,Palavra,Ocorrências
33,inteligência,9
21,emocional,4
24,expressão,3
71,tipo,3
16,conceito,2


#### TF-IDF

In [18]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<99x78 sparse matrix of type '<class 'numpy.float64'>'
	with 96 stored elements in Compressed Sparse Row format>

In [19]:
tvec = TfidfVectorizer(min_df=.0025, max_df=.1, ngram_range=(1,2))
tvec_weights = tvec.fit_transform(bowA)
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'Palavra': tvec.get_feature_names(), 'Frequencia': weights})
weights_df.sort_values(by='Frequencia', ascending=False).head(10)

,Palavra,Frequencia
33,inteligência,0.090909
21,emocional,0.040404
24,expressão,0.030303
71,tipo,0.030303
16,conceito,0.020202
53,pois,0.020202
58,referem,0.020202
52,pessoas,0.010101
57,refere,0.010101
56,recém,0.010101
